<a href="https://colab.research.google.com/github/AdvancedCRD/any-csv-preview/blob/main/01_any_csv_preview_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# IDENTIFY PROGRAMMER & GOAL (generic CSV → print first 10 rows)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
programmer_name = "CRD Student"
print(f"Programmer Name: {programmer_name}\nProgram Goal: Read ANY CSV and print the first five rows.")

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# IMPORT COMMON LIBRARIES (brief purpose notes)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import pandas as pd                  # tabular data (DataFrame)
import matplotlib.pyplot as plt      # plotting (not used here, ready for later)
import sys, subprocess               # runtime info + install third-party libs if missing
import math                          # numeric utilities (ready for later)

# Ensure Rich is available (pretty tables in Colab/Jupyter); auto-install if needed
try:
    from rich.console import Console
    from rich.table import Table
    from rich import box
    from rich.text import Text
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rich"])
    from rich.console import Console
    from rich.table import Table
    from rich import box
    from rich.text import Text

# (Optional) Mount Google Drive in Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# CONFIG: set your CSV path (works for ANY dataset)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
CSV_PATH = "/content/drive/My Drive/ACRD/Organ Transplantation/Overview2020-2025.csv"   # ← change this

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# RICH TABLE HELPER (generic, works for any DataFrame)
# - Right-align numeric columns; add thousands separators for large numbers
# - If a "Year" column exists and is numeric, show it without commas
# - Display "." for missing values (display only; does not modify the DataFrame)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MISSING_SYMBOL = "."
console = Console(force_jupyter=True)
TEAL = "#008b8b"

def df_to_rich_table(df: pd.DataFrame, title: str | Text = "") -> Table:
    t = Table(title=title, box=box.SIMPLE_HEAVY, show_header=True,
              header_style=f"bold {TEAL}")
    numeric_cols = {c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])}
    has_year_plain = ("Year" in df.columns) and ("Year" in numeric_cols)

    for c in df.columns:
        t.add_column(str(c), justify=("right" if c in numeric_cols else "left"),
                     overflow="fold", no_wrap=False)

    for row in df.itertuples(index=False, name=None):
        cells = []
        for val, col in zip(row, df.columns):
            if pd.isna(val):
                cells.append(MISSING_SYMBOL)
                continue
            if has_year_plain and col == "Year":
                # show Year without separators
                try:
                    cells.append(f"{int(val)}")
                except Exception:
                    cells.append(str(val))
            elif col in numeric_cols:
                v = float(val)
                v = int(v) if v.is_integer() else v
                cells.append(f"{v:,}" if abs(v) >= 1000 else f"{v}")
            else:
                cells.append(str(val))
        t.add_row(*cells)
    return t

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# READ ANY CSV → PRINT FIRST FIVE ROWS
# - thousands="," parses "12,345" as numbers automatically (if present)
# - na_values handles common missing tokens (incl. ".")
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
df = pd.read_csv(
    CSV_PATH,
    na_values=[".", "NA", "N/A", "", "null", "Null", "NaN"],
    keep_default_na=True,
    thousands=","     # if the file uses commas as thousands separators
)

console.print(
    df_to_rich_table(
        df.head(10),
        title=Text("Preview: first 10 rows", style=f"bold {TEAL}")
    )
)
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]:,} columns")


Programmer Name: CRD Student
Program Goal: Read ANY CSV and print the first five rows.
Mounted at /content/drive


                                              Preview: first 10 rows                                               
                                                                                                                   
                                                                                         Number of   Number of     
                        Number of      Number of                                          deceased   living donor  
                        deceased       living organ       Number of      Number of     donor organ   organ         
                        organ donors   donors            candidates   registration      transplant   transplant    
  Year   Organ          recovered      recovered              added        s added      recipients   recipients    
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  2020   Heart          3761                  .               4,532          4,594           3,647          .      
  2020   Kidney         11925          5234                  36,153         37,825          17,574   5232          
  2020   Kidney-Pancr          .              .               1,275          1,314             827          .      
         eas                                                                                                       
  2020   Liver          9210           491                   12,609         13,019           8,291   491           
  2020   Lung           2578                  .               2,656          2,675           2,536          .      
  2020   Pancreas       1266                  .                 446            470             134          .      
  2020   All            12588          5726                  55,057         59,897          31,778   5723          
  2021   Heart          3900                  .               4,948          5,008           3,806          .      
  2021   Kidney         13215          5971                  40,081         42,130          18,684   5969          
  2021   Kidney-Pancr          .              .               1,434          1,488             820          .      
         eas

Shape: 42 rows × 8 columns
